# 1.0 Extract movement and track positions over time.

For each video we use YOLOv8 to extract movement data as a set of body keypoints and use its `model.track` method to track individuals over time.


# 1.1 Video pose estimation with Ultralytics YOLO
[Ultralytics](https://github.com/ultralytics/ultralytics) is a commercially maintained version of the YOLO object recognition model. [Yolov7](https://github.com/WongKinYiu/yolov7) introduced pose estimation and v8 improves the models and makes everything much more user-friendly. The current version is YOLOv11. It can be installed as a package

* Pip : `pip install ultralytics`
* Conda : `conda install -c conda-forge ultralytics`

## 1.2 Object tracking 

Since YOLOv8, it also comes with a `model.track` method. This aims to keep track of all identified objects over the course of a video. Let's make use of that to track individuals over time. 

This is pretty easy instead of calling 
`results = model(video_path, stream=True)`

we can call
`results = model.track(video_path, stream=True)`

https://docs.ultralytics.com/modes/track/#persisting-tracks-loop

In [2]:
import os
import sys

project_root = os.path.join("..")
sys.path.append(project_root)

# Add debug prints to help diagnose the issue
print(f"Current working directory: {os.getcwd()}")


import pandas as pd
import numpy as np

# Now import project modules
from src.utils.io_utils import getProcessedVideos, saveProcessedVideos
from src.processors.video_processor import videotokeypoints
from src.models.keypoints import get_keypoint_columns
from src.utils.keypoint_utils import normalize_keypoints
from src.main import process_all_videos

Current working directory: c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code


In [3]:
# Add these to your imports
from src.config import PATH_CONFIG
from src.utils.notebook_utils import display_config_info, ensure_dir_exists

# Get paths from config
videos_in = PATH_CONFIG['videos_in']
data_out = PATH_CONFIG['data_out']

# Ensure output directory exists
if ensure_dir_exists(data_out):
    print(f"Created output directory: {data_out}")

# Display configuration information
display_config_info(videos_in, data_out, "Processing Configuration")



## Processing Configuration
    
| Configuration | Value | Status |
|---------------|-------|--------|
| Input Videos | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\LookitLaughter.test` | ✅ exists |
| Output Data | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\data\1_interim` | ✅ exists |
| Video Count | 54 videos | |

You can change these paths by modifying the `PATH_CONFIG` in `src/config.py` 
or by overriding them in this notebook.


In [5]:

metadata_file = "_LookitLaughter.test.xlsx"

#get metadata from excel file
metadata = pd.read_excel(os.path.join(videos_in, metadata_file))
metadata.head()

,VideoID,Consent,Sharing,ParentID,ChildID,Age,Gender,PregnancyLength,Languages,JokeOrder,...,Withdrawal,DatabraryShare,JokeType,HowFunny,LaughYesNo,JokeNum,JokeRep,JokeTake,Child,Parent
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,"['Peekaboo', 'NomNomNom', 'ThatsNotAHat', 'Tea...",...,False,yes,Peekaboo,Slightly funny,No,1,2,1,LEFT,RIGHT
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,"['Peekaboo', 'NomNomNom', 'ThatsNotAHat', 'Tea...",...,False,yes,Peekaboo,Slightly funny,No,1,3,1,LEFT,RIGHT
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,"['Peekaboo', 'NomNomNom', 'ThatsNotAHat', 'Tea...",...,False,yes,NomNomNom,Funny,No,2,1,1,LEFT,RIGHT
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,"['Peekaboo', 'NomNomNom', 'ThatsNotAHat', 'Tea...",...,False,yes,NomNomNom,Slightly funny,No,2,2,1,LEFT,RIGHT
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,accepted,public,5ZQUSU,2UWdXP,233,f,38 weeks,en,"['Peekaboo', 'NomNomNom', 'ThatsNotAHat', 'Tea...",...,False,yes,NomNomNom,Slightly funny,No,2,3,1,LEFT,RIGHT


In [6]:
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

Found existing processed_videos.csv with 54 rows.


,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Speech.file,Diary.file,Diary.when,LastError,annotatedVideo,annotated.when,FrameCount,Keypoints.normed,Face_Processing_Complete,Faces.normed
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,NaN,14.232999,...,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2025-04-03 20:16:15,NaN,NaN,NaN,216,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,True,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,NaN,14.263979,...,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2025-04-03 20:16:17,NaN,NaN,NaN,150,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,True,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,NaN,12.275790,...,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2025-04-03 20:16:18,NaN,NaN,NaN,89,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,NaN,13.920731,...,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2025-04-03 20:16:19,NaN,NaN,NaN,95,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,NaN,14.010793,...,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2025-04-03 20:16:20,NaN,NaN,NaN,132,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...


In [5]:

# Option 1: Process videos using the refactored functions
forcemetadata = False
forceprocess = False 

# Process all videos - keep this line the same
process_all_videos(videos_in, data_out, metadata_file, forcemetadata, forceprocess)

Processing video: 2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4
Found existing processedvideos.xlsx with 0 rows.
Extracting keypoints from 2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4...

video 1/1 (frame 1/216) c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code\..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 480x640 1 person, 20.9ms
video 1/1 (frame 2/216) c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code\..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 480x640 1 person, 7.0ms
video 1/1 (frame 3/216) c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code\..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 480x640 1 person, 6.0ms
video 1/1 (frame 4/216) c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code\..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 480x640 1 person, 6.0ms
video 1/1 (frame 5/216) c:\Users\caspar\OneDrive\LegoGPI\babyjokes\code\..\LookitLaughter.test\2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4: 480x640 1 person, 6.0ms
video 1/

In [6]:
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

Found existing processedvideos.xlsx with 54 rows.


,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Faces.file,Speech.when,Speech.file,Diary.file,Diary.when,LastError,annotatedVideo,annotated.when,FrameCount,Keypoints.normed
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,NaN,14.232999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,NaN,14.263979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,NaN,12.275790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,NaN,13.920731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,NaN,14.010793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...


## Create a set of normalised keypoint.csv. 

For modelling we want all movement data in standardised numerical format.
So 
1. Normalise x, y coordinates. Every coordinate is scaled to the range [0,1] where 1  is (framewidth, frameheight) respectively
2. We overwrite the 'person' column ["child","adult"] with numerical values [0,1] taken from 'index' column.

In [7]:
# The normalization is now handled in the main process_all_videos function,
# but we can also do it separately for each video if needed:

processedvideos = getProcessedVideos(data_out)

for index, row in processedvideos.iterrows():
    if pd.isnull(row.get("Keypoints.normed")) or not os.path.exists(row.get("Keypoints.normed", "")):
        print(f"Normalizing keypoints for {row['VideoID']}")
        from src.main import normalize_and_save_keypoints
        normalize_and_save_keypoints(row.to_dict(), data_out)
    else:
        print(f"Already normalized {row['VideoID']}")

# Refresh the dataframe to see the updated values
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

Found existing processedvideos.xlsx with 54 rows.
Already normalized 2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4
Already normalized 2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4
Already normalized 2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4
Already normalized 2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4
Already normalized 2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4
Already normalized 2UWdXP.joke3.rep2.take1.ThatsNotAHat_h265.mp4
Already normalized 2UWdXP.joke3.rep3.take1.ThatsNotAHat_h265.mp4
Already normalized 2UWdXP.joke4.rep1.take1.TearingPaper_h265.mp4
Already normalized 2UWdXP.joke4.rep2.take1.TearingPaper_h265.mp4
Already normalized 2UWdXP.joke4.rep3.take1.TearingPaper_h265.mp4
Already normalized 2UWdXP.joke5.rep1.take1.ThatsNotACat_h265.mp4
Already normalized 2UWdXP.joke5.rep2.take1.ThatsNotACat_h265.mp4
Already normalized 2UWdXP.joke5.rep3.take1.ThatsNotACat_h265.mp4
Already normalized 3dC3SQ.joke1.rep1.take1.TearingPaper_h265.mp4
Already normalized 3dC3SQ.joke1.rep2.take1.TearingPaper_h

,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Faces.file,Speech.when,Speech.file,Diary.file,Diary.when,LastError,annotatedVideo,annotated.when,FrameCount,Keypoints.normed
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,NaN,14.232999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,NaN,14.263979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,NaN,12.275790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,NaN,13.920731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,NaN,14.010793,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...
